In [1]:
import pandas as pd
import numpy as np
from os import path

# custom scripts
from scripts.dataset_io import getDatasetFilePath, getDataset

### 1 - Globals

In [2]:
dataDir: str = '../data'
countryName: str = 'Portugal'

settings: 'dict[str, any]' = {
	'filePath': getDatasetFilePath(
		dataDir, 
		countryName, 
		fallback='CPITimeSeries'
	),
	'countryName': countryName,
	'indicator': 'PCPI_IX',
}

### 2 - Extract relevant info + Data Understanding

In [3]:
def understandData(df: pd.DataFrame) -> None :
  print('DATAFRAME INFO')
  print('------')
  print('df shape:', df.shape)
  print('------')
  print('df head: \n', df.head())
  print('------')
  print('df dtypes:\n', df.dtypes)
  print('------')
  print('df describe: \n', df.describe())
  print('------')
  print('df info: \n', df.info())

In [4]:
df: pd.DataFrame = getDataset(dataDir, settings)

Reading from ../data/CPITimeSeries.csv ...
Done!


In [6]:
df

,cpi,year,month
0,42.432,1990,1
1,43.378,1990,2
2,43.632,1990,3
3,44.135,1990,4
4,44.603,1990,5
...,...,...,...
95,68.913,1997,12
96,69.006,1998,1
97,68.975,1998,2
98,69.161,1998,3
